In [7]:
import pandas as pd
import numpy as np
import re
import sys
from sklearn import *
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer 
from sklearn.ensemble import RandomForestClassifier

In [57]:
#Loading dataset
data_path = r"C:/Users/eunic/OneDrive/Desktop/trainingObamaRomneytweets.xlsx"
data_romney = pd.read_excel(data_path, sheetname = 'Romney')

data_path_test = r"C:/Users/eunic/OneDrive/Desktop/testObamaRomneytweets.xlsx"
data_romney_test = pd.read_excel(data_path_test, sheetname = 'Romney')

# Removing rows with classes other than 1,-1,0
data_romney = data_romney[(data_romney['Class'].isin((1,-1,0)))]
data_romney_test = data_romney_test[(data_romney_test['Class'].isin((1,-1,0)))]

stop_words = set(stopwords.words('english')) 
ps = PorterStemmer() 
lemmatizer = WordNetLemmatizer() 
lmtzr = WordNetLemmatizer();
vectorizer = CountVectorizer()

#predefined function for cleaning the text
def clean_text(row):
    tweet = str(row)
    cleanr = re.compile('(</?[a-zA-Z]+>|https?:\/\/[^\s]*|(^|\s)RT(\s|$)|@[^\s]+|\d+)')
    cleantext = re.sub(cleanr, ' ', tweet)
    cleantext = re.sub('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)',' ',cleantext)
    cleantext = re.sub('[^\sa-zA-Z]+','',cleantext)
    cleantext = re.sub('\s+',' ',cleantext)       
    lower_case = cleantext.lower() 
    stemmedTweet = [ps.stem(word) for word in lower_case.split(" ")]
    stemmedTweet = " ".join(stemmedTweet)
    tweet = str(stemmedTweet)
    tweet = tweet.replace("'", "")
    tweet = tweet.replace("\"","")
    tweet = lmtzr.lemmatize(tweet)
    return tweet

#Calculating tf-idf for each tweet
def vectorize_data(tweets): 
    vector = vectorizer.fit_transform(tweets)
    return vector

def vectorize_data_test(tweets):
    vector = vectorizer.transform(tweets)
    return vector

#Extracting only the class and annotation from given file
new_data_romney= data_romney[['Class','Anootated tweet']]
new_data_romney_test = data_romney_test[['Anootated tweet']]

#Creating a column with pre processed tweets
new_data_romney['Cleaned_tweet'] = new_data_romney['Anootated tweet'].apply(clean_text)
new_data_romney_test['Cleaned_tweet'] = new_data_romney_test['Anootated tweet'].apply(clean_text)

# converting the tweets and class to list form
cleaned_romney_tweets = new_data_romney['Cleaned_tweet'].tolist()
cleaned_romney_tweets_test = new_data_romney_test['Cleaned_tweet'].tolist()
romney_class = new_data_romney['Class'].tolist()
#print(cleaned_romney_tweets_test[:10])

#Vectorizing the text data
vector_romney_tweets = vectorize_data(cleaned_romney_tweets)
vector_romney_tweets_test = vectorize_data_test(cleaned_romney_tweets_test)

C:\Users\eunic\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\eunic\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['insidi mitt romney s bain help philip morri get us high schooler hook on cigarett via ', ' you mean like romney cheat in primari ', ' mitt romney still doesnt believ that we have a black presid ', ' romney s tax plan deserv a nd look becaus he ha a secret one that differ than the one he been lie about ', 'hope romney debat prep w the same peopl as last time', 'want to know how mitt romney is go to be abl to cut by trillon dollar go here it explain everyth ', 'if romney win the presidenti elect the world realli end thi year ', ' romney s million job scam remind me of rip torn sell penni in the movi sick puppi ', ' mitt romney said that catch osama bin laden would be insignific and it not worth move heaven and earth ', 'pleas be mitt romney who is honey boo boo endors for presid video ']
  (0, 6079)	1
  (0, 945)	1
  (0, 3836)	1
  (0, 2554)	1
  (0, 4864)	1
  (0, 2481)	1
  (0, 6033)	1
  (0, 2178)	1
  (0, 3520)	1
  (0, 4084)	1
  (0, 2462)	1
  (0, 402)	1
  (0, 4716)	1
  (0, 3464)	1
  (0, 2

In [58]:
# Naive Bayes Classifier 
print("Naive Bayes Classifier - Multinomial")
model = naive_bayes.MultinomialNB()
folds = KFold(n_splits = 10, shuffle = True)

#Romney
predicts1= cross_val_predict(model, vector_romney_tweets, romney_class, cv = folds)
classes = [1,0,-1]
print(predicts1[:10])
accScore_romney = metrics.accuracy_score(romney_class,predicts1)

precision = metrics.precision_score(romney_class,predicts1,average=None,labels=classes)
recall = metrics.recall_score(romney_class,predicts1,average=None,labels=classes)
f1Score = metrics.f1_score(romney_class,predicts1,average=None,labels=classes)
print("Romney:  Acurracy: ",accScore_romney)
print("Precision :", precision)
print("Recall", recall)
print("F1-Score", f1Score, "\n")

Naive Bayes Classifier - Multinomial
[-1 -1 -1 -1  0 -1 -1 -1 -1 -1]
Romney:  Acurracy:  0.5846317280453258
Precision : [0.55508475 0.49852217 0.6122293 ]
Recall [0.3655814  0.30119048 0.83062565]
F1-Score [0.44083006 0.3755102  0.7048988 ] 

